In [1]:
# Imports
import ast
import pickle
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from ranx import Run, fuse, evaluate, Qrels


# Parte 1: Definições

Esta seção define como a fusão é realizada e testa o processo.

In [2]:
## Funções úteis nessa parte

# Corrige as labels de "x_y" para "x"
def correct_scores(documents_dict_str):
    documents_dict = ast.literal_eval(documents_dict_str)

    new_dict = {}
    for k, v in documents_dict.items():
        new_key = k.split("_")[1]
        new_dict[new_key] = v

    return new_dict

def filter_good_expansions(full_dataset, model=None):
    if model is None:
        grouped = full_dataset.groupby('query_idx')
        filtered_dataset = []

        for _, group in grouped:
            # Get the first row's values for specific columns
            first_row = group[['query_idx', 'exp_num_y', 'original_passage_scores', 'bm25_scores']].iloc[0].tolist()
            filtered_dataset.append(first_row)

            # Filter rows where 'label' is 1
            ones_label = group[group['label'] == 1]
            if not ones_label.empty:  # Only add if there's at least one matching row
                filtered_values = ones_label[['query_idx', 'exp_num', 'expansion_passage_scores', 'bm25_scores']].values.tolist()
                filtered_dataset.extend(filtered_values)  # Flatten and add to dataset

    else:
        clean_dataset = full_dataset[[ "relevant_count","spearman" , "words_similarity",  "expansion_idf", "expansion_idf_difference"]] 
        clean_dataset = clean_dataset.rename(columns={
                                                        'words_similarity': 'words_semantic_similarity',
                                                        'relevant_count': 'k_relevance_judgments',
                                                        'spearman':'spearman_rank_correlation'
                                                    })

        clean_dataset = clean_dataset.apply(pd.to_numeric)
        predictions = model.predict(clean_dataset)
        model_dataset = full_dataset.copy()
        model_dataset["predictions"] = predictions

        grouped = model_dataset.groupby('query_idx')
        filtered_dataset = []

        for _, group in grouped:
            # Get the first row's values for specific columns
            first_row = group[['query_idx', 'exp_num_y', 'original_passage_scores', 'bm25_scores']].iloc[0].tolist()
            filtered_dataset.append(first_row)

            # Filter rows where 'label' is 1
            ones_prediction = group[group['predictions'] == 1]
            if not ones_prediction.empty:  # Only add if there's at least one matching row
                filtered_values = ones_prediction[['query_idx', 'exp_num', 'expansion_passage_scores', 'bm25_scores']].values.tolist()
                filtered_dataset.extend(filtered_values)  # Flatten and add to dataset

    # Create DataFrame from the filtered dataset
    columns = ['query_idx', 'exp_num', 'passage_scores', 'bm25_scores']  # Update as per your columns
    return pd.DataFrame(filtered_dataset, columns=columns).sort_values('exp_num')




def fuse_expansions(dataset_to_fuse:pd.DataFrame, method:str, include_bm25:bool=False):

    if include_bm25 and ('bm25_scores' not in dataset_to_fuse.columns):
        print("A coluna bm25_scores do dataset nao foi encontrada.")
        return
    
    # Corrige chave das passagens  {'passage_172362': 0.3176,...} --> {'172362': 0.3176,...}
    filtered_df = dataset_to_fuse.copy()
    filtered_df["passage_scores"] = filtered_df["passage_scores"].apply(lambda documents_dict: correct_scores(documents_dict))
    if include_bm25:
        filtered_df["bm25_scores"] = filtered_df["bm25_scores"].apply(lambda documents_dict: correct_scores(documents_dict))


    grouped_df = filtered_df.groupby('query_idx')

    # Inicializa o conjunto total de fusões
    all_fused_runs = {}
    count = 0
    # Como temos expansões de queries distintas, temos que agrupar de acordo com a query de origem antes de fundir.
    # Posteriormente elas as fusões de cada agrupamento podem compor um conjunto total das fusões.
    for query_idx, group in grouped_df:

        count +=1
        # Cria as runs (a partir da query original + expansões referentes a ela)
        runs = {}
        for _, row in group.iterrows():
            runs[f"{query_idx}_exp_{row['exp_num']}"] = Run({str(query_idx): row["passage_scores"]})

        # Se for escolhido agregar os resultados da BM25
        if include_bm25:
            runs[f"{query_idx}_bm25_exp_{row['exp_num']}"] = Run({str(query_idx): row["bm25_scores"]})
                
        # Faz a fusão a partir dos runs
        fused_run = fuse(list(runs.values()), method=method) if len(runs) > 1 else list(runs.values())[0] 
        all_fused_runs.update(fused_run.to_dict())

    #print(f"tamanho do all_sused_runs {len(all_fused_runs)}, quando deveriam ser {count}")

    fused_runs = Run(all_fused_runs)

    #print(f"Tamanho do dataset de expansões usado: {len(filtered_df)}")
    #print(f"Número de queries após fusão: {len(fused_runs)}")
    return fused_runs

def evaluate_fusion(fused_runs:Run, relevance_map:dict, metrics:list):
    qrels_query_ids = set(relevance_map.keys())
    run_query_ids = set(fused_runs.keys())

    # Identificar diferenças
    missing_in_run = qrels_query_ids - run_query_ids
    missing_in_qrels = run_query_ids - qrels_query_ids

    #print("Faltando no Run:", missing_in_run)
    #print("Faltando no Qrels:", missing_in_qrels)

    run_dict = fused_runs.to_dict()

    qrels_query_ids = set(relevance_map.keys())
    run_query_ids = set(run_dict.keys())

    # Consultas faltantes no Run
    missing_in_run = qrels_query_ids - run_query_ids
    for query_id in missing_in_run:
        run_dict[query_id] = {}  # Adiciona consulta com resultados vazios (para não criar distorções)

    # Consultas extras no Run
    missing_in_qrels = run_query_ids - qrels_query_ids
    for query_id in missing_in_qrels:
        run_dict.pop(query_id)

    # Ordenar o Qrels (relevance_map) e os runs
    sorted_relevance_map = {query_id: relevance_map[query_id] for query_id in sorted(relevance_map.keys())}
    sorted_run_dict = {query_id: run_dict[query_id] for query_id in sorted(run_dict.keys())}
    fused_runs = Run(sorted_run_dict)
    relevance_map = Qrels(sorted_relevance_map)

    # Avalia as fusões de cada grupo de expansões
    return evaluate(
        sorted_relevance_map, 
        fused_runs,
        metrics=metrics,
        make_comparable=True  
    )


In [ ]:
## RECUPERA DADOS IMPORTANTES PARA A FUSÃO

# DATASET COM AS EXPANSÕES ENRIQUECIDAS
full_dataset = pd.read_csv(
    "../1_enrich_results/queries_train_judged_expanded_enriched.csv",
    sep='\t',
    index_col=0
)
print(f"queries originais mantidas até aqui: {full_dataset['query_idx'].nunique()}")

# QRELS (relevance judgments) do dataset MS MARCO dev judged
qrels = Qrels.from_ir_datasets("msmarco-passage/train/judged")

# MODELO ANTERIORMENTE TREINADO
model = xgb.XGBClassifier()
model.load_model("../3_xgboost_model/xgboost_model.json")


In [4]:
## RECONSTRÓI O MAPA DE RELEVÂNCIA
# O mapa de relevância foi salvo na etapa "create_folds", 
# Como o mesmo id da query original (query_idx) se repete para muitas expansões, foi criado um idx temporáro para elas
# porém, depois que a fusão é realizada, temos que transformar o mapa de relevância para refletir novamente o id original da query e calcular corretamente as métricas


with open(f"../../input_data/samples.pkl", "rb") as dataset_file:
    samples = pickle.load(dataset_file)
    samples = pd.DataFrame(samples)

relevance_map_path = "../../input_data/relevance_map.pkl"
with open(relevance_map_path, "rb") as relevances_file:
    data = pickle.load(relevances_file)

incorrect_relevance_map = {}
for text_idx, labels_ids in data.items():
    d = {}
    for label_idx in labels_ids:
        d[f"{label_idx}"] = 1.0
    incorrect_relevance_map[f"{text_idx}"] = d 

relevance_map = {}
for key, item in incorrect_relevance_map.items():
    new_key = samples.loc[samples['idx'] == int(key)]['query_exp_id'].values[0].split('_')[0]
    relevance_map[new_key] = item

#relevance_map

### **1.1** Avaliação contendo apenas as consultas originais

Apenas os registros que tem exp_num = 1, que são os originais

In [ ]:
grouped = full_dataset.groupby('query_idx')
filtered_dataset = {}

for query_idx, group in grouped:
    first_row = group[['query_idx', 'exp_num_y', 'original_passage_scores']].iloc[0]
    filtered_dataset[str(first_row['query_idx'])] = correct_scores(first_row['original_passage_scores'])
original_runs = Run(filtered_dataset)

original_eval_results = evaluate_fusion(original_runs, relevance_map, ["ndcg@1", "ndcg@5", "ndcg@10", "precision", "recall", "map"])
print("Combined Evaluation Results:", original_eval_results)

### **1.2** Fusão contendo todas as expansões consideradas boas

Faz o rank fusion usando informação a privilegiada do dataset (label)

In [ ]:
filtered_expansions_ideal = filter_good_expansions(full_dataset)
ideal_fused_runs = fuse_expansions(filtered_expansions_ideal, "mnz")

eval_results = evaluate_fusion(ideal_fused_runs, relevance_map, ["ndcg@1", "ndcg@5", "ndcg@10", "precision", "recall", "map"])
print("Combined Evaluation Results:", eval_results)

In [ ]:
# RESULTADO DA FUSÂO
ideal_fused_runs_dict = ideal_fused_runs.to_dict()
fused_dataset_ideal_filter = full_dataset[["query_idx", "query_original"]].copy().drop_duplicates().reset_index(drop=True)
fused_dataset_ideal_filter["fused_ranking"] = fused_dataset_ideal_filter["query_idx"].apply(lambda x: ideal_fused_runs_dict[str(x)] if str(x) in ideal_fused_runs_dict.keys() else {})
fused_dataset_ideal_filter.tail(5)

### **1.3** Fusão contendo as expansões boas identificadas pelo modelo anteriormente treinado

Faz o rank fusion usando o modelo treinado

In [ ]:
filtered_expansions_model = filter_good_expansions(full_dataset, model=model)
model_fused_runs = fuse_expansions(filtered_expansions_model, "mnz")
eval_results_model = evaluate_fusion(model_fused_runs, relevance_map, ["ndcg@1", "ndcg@5", "ndcg@10", "precision", "recall", "map"])
print("Combined Evaluation Results:", eval_results_model)

In [ ]:
# RESULTADO DA FUSÂO
model_fused_runs_dict = model_fused_runs.to_dict()
fused_dataset_model_filter = full_dataset[["query_idx", "query_original"]].copy().drop_duplicates().reset_index(drop=True)
fused_dataset_model_filter["fused_ranking"] = fused_dataset_model_filter["query_idx"].apply(lambda x: model_fused_runs_dict[str(x)] if str(x) in model_fused_runs_dict.keys() else {})
fused_dataset_model_filter.tail(5)

### Agrega um novo retriever (BM25) para as expansões consideradas boas pelo modelo

In [ ]:
filtered_expansions_model = filter_good_expansions(full_dataset, model=model)
model_fused_runs = fuse_expansions(filtered_expansions_model, "mnz", True)
eval_results_model = evaluate_fusion(model_fused_runs, relevance_map, ["ndcg@1", "ndcg@5", "ndcg@10", "precision", "recall", "map"])
print("Combined Evaluation Results:", eval_results_model)

-----------------

# Parte 2: Fusão e avaliação dos resultados

In [ ]:
# Os algoritmos de fusão utilizados aqui são aqueles que não requerem parâmetro de otimização
results={}
fusion_algorithms = ['min', 'med', 'anz', 'log_isr', 'bordafuse', 'condorcet', 'max', 'sum', 'mnz', 'isr']

for algorithm in fusion_algorithms:

    grouped = full_dataset.groupby('query_idx')
    filtered_dataset = {}
    for query_idx, group in grouped:
        first_row = group[['query_idx', 'exp_num_y', 'original_passage_scores']].iloc[0]
        filtered_dataset[str(first_row['query_idx'])] = correct_scores(first_row['original_passage_scores'])
    original_runs = Run(filtered_dataset)
    original_eval_results = evaluate_fusion(original_runs, relevance_map, ["ndcg@1", "ndcg@5", "ndcg@10", "precision", "recall", "map"])
    results['1-orign_' + algorithm] = original_eval_results

    ### Faz a expansão com info privilegiada
    ideal_filtered_expansions = filter_good_expansions(full_dataset)
    ideal_fused_runs = fuse_expansions(ideal_filtered_expansions, algorithm)
    ideal_eval_results = evaluate_fusion(ideal_fused_runs, relevance_map, ["ndcg@1", "ndcg@5", "ndcg@10", "precision", "recall", "map"])
    results['4-ideal_' + algorithm] = ideal_eval_results

    ### Faz a expansão com julgada pelo modelo
    model_filtered_expansions = filter_good_expansions(full_dataset, model=model)
    model_fused_runs = fuse_expansions(model_filtered_expansions, algorithm)
    model_eval_results_model = evaluate_fusion(model_fused_runs, relevance_map, ["ndcg@1", "ndcg@5", "ndcg@10", "precision", "recall", "map"])
    results['2-model_' + algorithm] = model_eval_results_model

    ### Faz a expansão com julgada pelo modelo acrescentando os resultados da bm25
    bm25_filtered_expansions = filter_good_expansions(full_dataset, model=model)
    bm25_fused_runs = fuse_expansions(bm25_filtered_expansions, "mnz", True)
    eval_results_bm25 = evaluate_fusion(bm25_fused_runs, relevance_map, ["ndcg@1", "ndcg@5", "ndcg@10", "precision", "recall", "map"])
    results['3-model+bm25_' + algorithm] = eval_results_bm25

    print(f"Expansões com {algorithm} realizada com sucesso!")


df_results = pd.DataFrame.from_dict(results, orient='index')
print(df_results)

In [ ]:
# Calcular as variações percentuais em relação à linha "1-orign"
df_plot = df_results.copy()

df_plot['algorithm'] = df_plot.index.str.split('_').str[1]
df_plot['prefix'] = df_plot.index.str.split('_').str[0]

baseline = df_plot[df_plot["prefix"] == "1-orign"].iloc[0]
df_plot["variation"] = df_plot.apply(
    lambda row: {
        col: ((row[col] - baseline[col]) / baseline[col]) * 100
        for col in ["ndcg@1", "ndcg@5", "ndcg@10", "precision", "recall", "map"]
    },
    axis=1,
)

# Plotar cada grupo separadamente como barras
grouped = df_plot.groupby("algorithm")

for algorithm, group in grouped:
    group = group.sort_values("prefix")
    metrics = ["ndcg@1", "ndcg@5", "ndcg@10", "precision", "recall", "map"]

    # Plotar como barras
    fig, ax = plt.subplots(figsize=(12, 7))
    group[metrics].T.plot(kind="bar", ax=ax, width=0.7)  # Aumenta a largura das barras

    # Adicionar os valores e variações acima das barras
    for i, prefix in enumerate(group["prefix"]):
        for j, metric in enumerate(metrics):
            value = group.iloc[i][metric]
            variation = group.iloc[i]["variation"][metric]
            ax.text(
                j + i * 0.27 - 0.3, value + 0.005,  # Ajustar posição
                f"{value:.4f} \n ({variation:.2f}%)",
                ha="center", va="bottom", fontsize=8
            )

    # Personalizar o gráfico
    plt.title(f"Resultados comparativos entre consulta original e fusão com uso do algoritmo {algorithm}")
    plt.xlabel("Métricas")
    plt.ylabel("Valores")
    plt.legend(title="Prefixos", labels=group["prefix"], loc="upper right")
    plt.xticks(rotation=45)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
for algorithm, group in grouped:
    if algorithm == "bordafuse":
        group = group.sort_values("prefix")
        print(group)

### Descrição das métricas utilizadas:

**NDCG** = mede a qualidade dos resultados levando em conta a posição dos itens relevantes. Resultados relevantes em posições mais altas contribuem mais para a pontuação. Pode considerar apenas as k primeiras posições.

**Precision** = Fração de instancias relevantes recuperadas, dentre todas as instâncias recuperadas. Responde a pergunta "Dentre os itens recuperados, quantos são relevantes?"

**Recall** = proporção de itens relevantes retornados em relação ao total de itens relevantes existentes no conjunto de dados. Não considera a posição dos itens, apenas a cobertura. Responde a pergunta "Dentre todos os itens relevantes, quantos foram recuperados?"

**MAP** = precisão ao longo do ranking ponderando pela posição de cada item relevante. É a média das precisões médias (AP) de várias consultas. Combina precisão e posição, sendo ideal para tarefas onde a ordem dos resultados relevantes é importante.

Todas as métricas estão disponíveis em: https://amenra.github.io/ranx/metrics/


--------------